In [1]:
!pip install instagramy

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
conda 4.10.3 requires ruamel_yaml_conda>=0.11.14, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Instagram Analytics 
This notebook uses [Instagramy](https://pypi.org/project/instagramy/) to pull data on a users last 12 posts and persiest it in a csv to give a longer term view of a users posts. 
Individual post data also be updated via the use of a post short code. 
E.g. the short code for this post https://www.instagram.com/p/CXs5VlKADW4/ is CXs5VlkADW4


In [2]:
import pandas as pd
import instagramy 
import plotly.express as px

In [5]:
# load old data if any exists 
old =pd.read_csv("data.csv")

In [6]:
# get user data
user = instagramy.InstagramUser("bennnnnnnnn9")

In [7]:
#Extract recent post form user data
posts =user.posts
rows = []
for post in posts:
    likes = post.likes
    if post.is_video:
        rows.append([post.shortcode, post.likes, post.taken_at_timestamp, post.video_view_count ])
    else:
        rows.append([post.shortcode, post.likes, post.taken_at_timestamp, None ])

df =pd.DataFrame(rows, columns=["Short", "Like", "Time","Views"])
df




,Short,Like,Time,Views
0,CXvd1xEA4yc,28,2021-12-21 21:33:53,77.0
1,CXs5VlKADW4,155,2021-12-20 21:39:33,891.0
2,CXpSvOjAXuw,62,2021-12-19 12:08:52,190.0
3,CXkx1dZAo4K,25,2021-12-17 17:58:38,78.0
4,CXilbokADhN,195,2021-12-16 21:32:32,918.0
5,CXVpDnqAhVa,84,2021-12-11 20:51:58,300.0
6,CXUvVQXgxeZ,130,2021-12-11 12:30:37,505.0
7,CXUWIj-hf-D,85,2021-12-11 08:48:32,350.0
8,CXQW3ITAL85,286,2021-12-09 19:37:44,1690.0
9,CXLcQPAARNU,85,2021-12-07 21:47:18,335.0


In [9]:
#add random viral postout side of recent 12 via short code

oneofpost = instagramy.InstagramPost("CXNj_AwgRmc")
rows =[]
if oneofpost.post_data["is_video"]:
    rows.append([oneofpost.post_data["shortcode"], oneofpost.number_of_likes, oneofpost.upload_time, oneofpost.post_data["video_view_count"]])
else:
    rows.append([oneofpost.post_data["shortcode"], oneofpost.number_of_likes, oneofpost.upload_time, None ])

dfone =pd.DataFrame(rows, columns=["Short", "Like", "Time","Views"])  
df = pd.concat([df,dfone])
df["Time"]=pd.to_datetime(df["Time"])
df["ratio"] = 100*(df["Like"]/df["Views"])
combined = pd.concat([df,old])
df2 = combined.drop_duplicates(subset = ["Short"],keep="first", ignore_index=True) 

# Visualisation 

In [21]:
# display posts over time coloured by like ratio
fig = px.bar(df2, x="Time", y="Like",color="ratio", color_continuous_scale="sunsetdark",  title="Likes per Post over time")
fig.show()

In [24]:
fig = px.violin(df2, y="Like", box=True, points="all", title="Like distribution" )
fig.show()

# Write dataframe to csv for enduring recored of older post

In [15]:
df2.to_csv("data.csv", index=False)